<a href="https://colab.research.google.com/github/jasreman8/Multi-Agent-System-Projects/blob/main/CrewAI_Multi_Agent_Employee_Onboarding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Objectives

- Understand Sequential Multi-Agent System Design: Grasp how to conceptualize and implement a workflow using multiple, specialized agents working in sequence.
- Manage Workflow State: Understand the concept of a shared state in a multi-step process and how to define it using TypedDict to track information passed between agents.
- Construct Agentic Graphs with LangGraph:
Define individual tasks as nodes within a StateGraph.
Connect these nodes to create a specific execution flow (e.g., sequential).

# Business Scenario

**Automated Employee Onboarding Process**

In this notebook, we implement an automated system for onboarding new employees in a company. The onboarding process typically involves several departments and a sequence of tasks that must be completed before a new hire's first day.

Manual employee onboarding can be:
- Time-consuming: HR, IT, and hiring managers spend significant time on repetitive tasks.
- Error-prone: Manual data entry and task tracking can lead to mistakes, such as incorrect account setup or missed steps.
- Inconsistent: The new hire experience can vary depending on who is handling the onboarding.
- Delayed: Bottlenecks in one department can delay the entire process, impacting the new hire's productivity from day one.


Solution:

This system uses a chain of three specialized AI agents to automate key onboarding tasks:
- HR Coordinator Agent: Creates the employee record in the HR system and initiates a background check.
- IT Provisioner Agent: Sets up IT accounts (email, SSO) and arranges for necessary hardware.
- Welcome & Training Agent: Generates a welcome email and schedules initial onboarding training sessions.

By automating these steps, the aim is to make the onboarding process more efficient, consistent, and reduce the manual burden on company staff, allowing new hires to be productive faster.

# Setup

!pip install -q openai==1.66.3 \
                crewai==0.114.0

In [2]:
import os, json, random, datetime

from typing import List, Optional
from pydantic import BaseModel, Field

from crewai import LLM, Agent, Task, Crew, Process
from crewai.tools import tool

from google.colab import userdata

In [3]:
openai_api_key = userdata.get('OPEN_API_KEY')

os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['OPENAI_API_BASE'] = "https://aibe.mygreatlearning.com/openai/v1"

llm = LLM(
    model='gpt-4o-mini',
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    api_key=openai_api_key,
    temperature=0
)

# Implementation Plan

The system is designed as an automated assembly line for onboarding new employees, with different AI "specialists" handling distinct parts of the process.

- Prepare the Toolbox: First, we create a set of specific functions, our "tools." Each tool performs a single, well-defined action that's part of the onboarding process. For example, one tool creates an employee record in an HR system, another provisions an IT account, and another generates a welcome email. These tools are like specialized instruments that our AI specialists will use.
- Assemble the Specialist Team (Agents): We then define several AI "specialists" (agents). Each specialist has a specific role (e.g., "HR Coordinator," "IT Provisioner," "Welcome Coordinator") and a clear goal. We equip each specialist with the particular tools they need to do their job. For instance, the HR Coordinator gets the tools for creating records and initiating background checks.
- Design the Workflow (Tasks): Next, we break down the entire onboarding process into a series of sequential "tasks." Each task is assigned to one of our specialists. We provide a clear description of what needs to be done in each task, what information it needs from previous tasks (if any), and what the expected outcome or output should look like. For example, the IT Provisioning task can only start after the HR Coordinator has successfully created an employee ID.
- Form the Onboarding Crew and Kick Off: Finally, we gather all our specialists and their assigned tasks into an "onboarding crew." We instruct this crew to execute the tasks one after another, in a predefined order. We start the process by giving the initial details of the new hire (name, email, job title, etc.) to the first specialist in the chain. The crew then works through the tasks automatically, with the output of one task becoming the input for the next, until the entire onboarding sequence is complete. The final consolidated result of all these steps is then provided.

This approach ensures that each phase of the onboarding is handled methodically by the appropriate specialist using the correct tools, leading to an efficient and organized process.

# Tool Definitions

This section defines a set of "tools" that the AI agents can use to perform specific actions. In a real-world scenario, these tools might interact with actual databases, APIs, or external services. Here, they are simulated functions.

- Purpose of Tools: Tools provide agents with capabilities beyond simple text generation. They allow agents to interact with their environment, retrieve information, or execute operations.
- Tool Functionality: Each function simulates a specific onboarding step:
    - `create_employee_record_api`: Simulates creating an HR record and generating an employee ID.
    - `initiate_background_check_api`: Simulates starting a background check process.
    - `provision_account_api`: Simulates creating IT user accounts (email, username).
    - `assign_hardware_api`: Simulates assigning standard company hardware based on job title.
    - `generate_welcome_package`: Simulates creating the content for a welcome email.
    - `schedule_training_api`: Simulates scheduling standard onboarding training sessions.
- Output: Each tool returns a JSON string, which is a common format for API responses and allows structured data to be passed back to the agent. The print statements inside each tool are for demonstration purposes, showing when a tool is "called."

In [4]:
# Tool 1: Create Employee Record
@tool("create_employee_record")
def create_employee_record_api(name: str, email: str, job_title: str, start_date: str) -> str:
    """
    Creates a new employee record in the main HR database system.
    Returns a JSON string with the new employee ID and confirmation message.
    """
    print(f"--- HR Tool: Creating record for {name} ({email}), Title: {job_title}, Start: {start_date} ---")
    employee_id = f"EMP{random.randint(10000, 99999)}"
    result = {
        "success": True,
        "employee_id": employee_id,
        "message": f"Employee record created successfully for {name} with ID {employee_id}."
    }
    return json.dumps(result)

In [5]:
# Tool 2: Initiate Background Check
@tool("initiate_background_check")
def initiate_background_check_api(employee_id: str, full_name: str, email: str) -> str:
    """
    Initiates a standard background check process for the given employee ID via a third-party service.
    Returns a JSON string with a tracking ID and confirmation message.
    """
    print(f"--- HR Tool: Initiating background check for {full_name} (ID: {employee_id}, Email: {email}) ---")
    check_tracking_id = f"BGC-{random.randint(100000, 999999)}"
    result = {
        "success": True,
        "background_check_tracking_id": check_tracking_id,
        "message": f"Background check initiated for {employee_id}. Tracking ID: {check_tracking_id}."
    }
    return json.dumps(result)

In [6]:
# Tool 3: Provision IT Account
@tool("provision_it_account")
def provision_account_api(employee_id: str, full_name: str, job_title: str) -> str:
    """
    Provisions standard IT accounts (email, SSO, basic system access) for the new employee.
    Generates a username based on the name.
    Returns a JSON string with username, temporary password (placeholder), and confirmation.
    """
    print(f"--- IT Tool: Provisioning accounts for {full_name} (ID: {employee_id}), Title: {job_title} ---")
    name_parts = full_name.lower().split()
    username = f"{name_parts[0][0]}{name_parts[-1]}"
    temp_password = f"ChangeMe{random.randint(1000,9999)}!"
    email_address = f"{username}@example-company.com"
    result = {
        "success": True,
        "username": username,
        "email": email_address,
        "temp_password_notice": "Temporary password will be securely communicated separately.",
        "message": f"Standard accounts provisioned for {employee_id} with username '{username}'."
    }
    return json.dumps(result)

In [7]:
# Tool 4: Assign Hardware
@tool("assign_hardware")
def assign_hardware_api(employee_id: str, job_title: str, shipping_address: str) -> str:
    """
    Assigns standard hardware (laptop model, monitor) based on job title and queues it for shipment.
    Uses a predefined mapping of titles to hardware sets. Requires the employee's shipping address.
    Returns a JSON string with assigned hardware details and shipment tracking ID (placeholder).
    """
    print(f"--- IT Tool: Assigning hardware for {employee_id}, Title: {job_title}, Addr: {shipping_address[:20]}... ---")
    hardware_set = "Standard Laptop + Monitor"
    if "engineer" in job_title.lower() or "developer" in job_title.lower():
        hardware_set = "High-Performance Laptop + Dual Monitors"
    elif "designer" in job_title.lower():
        hardware_set = "MacBook Pro + High-Res Monitor"
    shipment_tracking = f"HW-SHIP-{random.randint(100000, 999999)}"
    result = {
        "success": True,
        "assigned_hardware": hardware_set,
        "shipment_tracking_id": shipment_tracking,
        "shipping_to": shipping_address,
        "message": f"Hardware '{hardware_set}' assigned to {employee_id} and queued for shipment."
    }
    return json.dumps(result)

In [8]:
# Tool 5: Generate Welcome Package
@tool("generate_welcome_package")
def generate_welcome_package(employee_id: str, full_name: str, job_title: str, start_date: str, first_day_contact: str, username: Optional[str]) -> str:
    """
    Generates the content for a personalized welcome email package.
    Includes first-day instructions, IT login info placeholder, and contact person details.
    Returns a JSON string containing the formatted email subject and body.
    """
    print(f"--- Welcome Tool: Generating package for {full_name} (ID: {employee_id}), Start: {start_date} ---")
    subject = f"Welcome to Example Company, {full_name}!"
    body = (
        f"Dear {full_name},\n\n"
        f"We are thrilled to welcome you to Example Company as our new {job_title}! "
        f"Your first day is scheduled for {start_date}.\n\n"
        f"Please plan to arrive by 9:00 AM. Your primary contact for the first day will be {first_day_contact}.\n\n"
        f"Your IT accounts have been set up. Your username is: {username or 'Details pending'}. "
        f"You will receive separate instructions on setting your initial password.\n\n"
        f"We have also scheduled some introductory training sessions for you (details below or in a separate calendar invite).\n\n"
        f"We look forward to having you on the team!\n\n"
        f"Best regards,\nExample Company HR"
    )
    result = {
        "success": True,
        "email_subject": subject,
        "email_body": body,
        "message": f"Welcome package content generated for {employee_id}."
    }
    return json.dumps(result)

In [9]:
# Tool 6: Schedule Training
@tool("schedule_training")
def schedule_training_api(employee_id: str, start_date: str) -> str:
    """
    Schedules the new employee for standard onboarding training sessions (e.g., HR Policies, IT Security)
    based on their start date. Returns a JSON string listing the scheduled sessions and dates/times.
    """
    print(f"--- Welcome Tool: Scheduling training for {employee_id}, starting {start_date} ---")
    try:
        s_date = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
    except ValueError:
        s_date = datetime.date.today() + datetime.timedelta(days=1) # Fallback

    training_schedule = [
        {"session": "HR Policies Onboarding", "datetime": (s_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d 10:00 AM")},
        {"session": "IT Security Awareness", "datetime": (s_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d 02:00 PM")},
        {"session": "Benefits Enrollment", "datetime": (s_date + datetime.timedelta(days=2)).strftime("%Y-%m-%d 11:00 AM")}
    ]
    result = {
        "success": True,
        "scheduled_trainings": training_schedule,
        "message": f"Standard onboarding training scheduled for {employee_id}."
    }
    return json.dumps(result)

# Output Definitions

This section defines Pydantic models that specify the expected structure and data types for the outputs of various tasks.Pydantic ensures that the data produced by an agent/task conforms to the defined schema. If an agent produces output that doesn't match, Pydantic will raise an error, helping to catch issues early.

- `HROutput`: Defines the expected output structure from the HR Coordinator's task. It includes crucial information like `employee_id` and status messages, as well as original input details that need to be passed to subsequent tasks.
- `ITOutput`: Defines the structure for the IT Provisioner's task output, including IT-specific details and again, passthrough data.
- `WelcomeTrainingInfo` & `WelcomeEmailContent`: These are nested models used within `WelcomeOutput` to structure complex data like training schedules and email content.
- `WelcomeOutput`: Defines the structure for the final Welcome & Training task, consolidating all welcome-related information.
- Passing Data: Notice fields like original_name, original_email, etc. These are included in the Pydantic models to explicitly ensure that initial data or data generated by early tasks is passed through the chain to later tasks that might need it.

In [10]:
class HROutput(BaseModel):
    """Structured output for the HR Coordination task."""
    employee_id: Optional[str] = Field(..., description="The unique ID generated for the new employee.")
    hr_record_status: Optional[str] = Field(..., description="Status message confirming HR record creation.")
    background_check_status: Optional[str] = Field(..., description="Status message confirming background check initiation.")
    # Pass original details needed by later tasks
    original_name: Optional[str] = Field(description="Original full name provided.")
    original_email: Optional[str] = Field(description="Original email provided.")
    original_job_title: Optional[str] = Field(description="Original job title provided.")
    original_start_date: Optional[str] = Field(description="Original start date provided.")
    original_shipping_address: Optional[str] = Field(description="Original shipping address provided.")

In [11]:
class ITOutput(BaseModel):
    """Structured output for the IT Provisioning task."""
    employee_id: Optional[str] = Field(description="The unique ID of the employee (passed from HR task).")
    it_account_status: Optional[str] = Field(..., description="Status message confirming IT account provisioning.")
    username: Optional[str] = Field(..., description="The generated username for the employee.")
    hardware_status: Optional[str] = Field(..., description="Status message confirming hardware assignment.")
    assigned_hardware: Optional[str] = Field(..., description="Description of the hardware assigned.")
    shipment_tracking_id: Optional[str] = Field(..., description="The tracking ID for the hardware shipment.")
    # Pass necessary details forward
    original_name: Optional[str] = Field(description="Original full name provided.")
    original_job_title: Optional[str] = Field(description="Original job title provided.")
    original_start_date: Optional[str] = Field(description="Original start date provided.")

In [12]:
class WelcomeTrainingInfo(BaseModel):
    """Represents a single scheduled training session."""
    session: str = Field(..., description="Name of the training session.")
    datetime: str = Field(..., description="Date and time of the session (YYYY-MM-DD HH:MM AM/PM).")

class WelcomeEmailContent(BaseModel):
    """Represents the content of the welcome email."""
    subject: str = Field(..., description="The subject line of the welcome email.")
    body: str = Field(..., description="The full body content of the welcome email.")

class WelcomeOutput(BaseModel):
    """Structured output for the Welcome & Training task."""
    employee_id: Optional[str] = Field(description="The unique ID of the employee.")
    username: Optional[str] = Field(description="The employee's username.")
    welcome_package_status: Optional[str] = Field(..., description="Status message confirming welcome package generation.")
    welcome_email_content: Optional[WelcomeEmailContent] = Field(..., description="The generated subject and body of the welcome email.")
    training_schedule_status: Optional[str] = Field(..., description="Status message confirming training scheduling.")
    scheduled_trainings: Optional[List[WelcomeTrainingInfo]] = Field(..., description="List of scheduled training sessions.")
    final_onboarding_status: str = Field(description="Overall status of the onboarding process after this step.")

# Agent Definitions

This section defines the AI agents that will perform the work. Each agent is configured with a specific role, goal, backstory, the LLM it uses, and the tools it has access to.
- Purpose of Agents: Agents are the autonomous entities in Crew AI that can reason, plan, and execute tasks using the tools provided.
- Agent Configuration:
    - `hr_coordinator`: This agent is responsible for the initial HR-related tasks. It has access to tools for creating employee records and initiating background checks.
    - `it_provisioner`: This agent handles IT setup. It can provision accounts and assign hardware.
    - `welcome_coordinator`: This agent manages the final steps of welcoming the employee, generating welcome materials, and scheduling training.
- Key Agent Parameters:
    - `role`: A concise description of the agent's persona (e.g., 'HR Coordinator'). This helps the LLM adopt the correct mindset.
    - `goal`: A clear statement of what the agent is trying to achieve.
    - `backstory`: Provides context and motivation for the agent, further guiding its behavior.
    - `llm`: Specifies the language model instance (created earlier) that this agent will use for its reasoning and decision-making.
    - `tools`: A list of tool functions that this specific agent is allowed to use. An agent can only use tools explicitly assigned to it.
    - `allow_delegation=False`: This setting prevents the agent from delegating its task to another agent. In this sequential setup, delegation is not needed.
    - `verbose=True`: Enables detailed logging of the agent's thought process and actions, which is useful for debugging and understanding how the agent works.
    - `memory=True`: Allows the agent to maintain a short-term memory of its interactions within the current task execution, which can help it make more contextually relevant decisions.

In [13]:
hr_coordinator = Agent(
    role='HR Coordinator',
    goal='Process new hire information to create an official employee record and initiate a background check.',
    backstory=(
        "You are the first point of contact in the automated employee onboarding process. "
        "Your primary responsibility is to use the provided tools to accurately create the employee's core HR record "
        "and kick off the mandatory background check. You must extract the new employee ID for subsequent steps."
    ),
    llm=llm,
    tools=[create_employee_record_api, initiate_background_check_api],
    allow_delegation=False,
    verbose=True,
    memory=True
)


In [14]:
it_provisioner = Agent(
    role='IT Provisioner',
    goal='Set up necessary IT accounts and assign standard hardware for the new employee.',
    backstory=(
        "You are responsible for the technical setup of a new employee. Using the employee ID and job details "
        "provided by HR, you provision standard IT accounts (like email and SSO) and determine the appropriate "
        "hardware package (laptop, monitors). You need the shipping address to queue the hardware shipment."
    ),
    llm=llm,
    tools=[provision_account_api, assign_hardware_api],
    allow_delegation=False,
    verbose=True,
    memory=True
)

In [15]:
welcome_coordinator = Agent(
    role='Welcome and Training Coordinator',
    goal='Generate a welcome package and schedule initial onboarding training sessions for the new employee.',
    backstory=(
        "You handle the final automated steps of onboarding. Your tasks are to create a personalized welcome email "
        "containing first-day information (using details like name, title, start date, username, and a placeholder contact) "
        "and schedule the employee for standard company training sessions based on their start date."
    ),
    llm=llm,
    tools=[generate_welcome_package, schedule_training_api],
    allow_delegation=False,
    verbose=True,
    memory=True
)

# Task Definitions

This section defines the individual tasks that make up the onboarding workflow. Each task is assigned to an agent and describes what needs to be done.

- Purpose of Tasks: Tasks are the units of work in Crew AI. They provide agents with specific instructions and goals.
- Sequential Execution and Context:
    - The `hr_task` is the first task. Its description uses placeholders like {name}, {email} which will be filled from the initial input provided when the crew is "kicked off."
    - The it_task depends on the `hr_task`. The `context=[hr_task]` parameter means that the output of hr_task will be available as context to the `it_provisioner` agent when it executes `it_task`. The description for `it_task` explicitly mentions that it will receive data (like employee_id) from the HR processing results.
    - Similarly, welcome_task depends on it_task (context=[it_task]) and will receive its output.
    - This context mechanism is how data flows sequentially from one task to the next.
- Importance of description and expected_output:
    - `description`: This is a detailed prompt for the agent. It tells the agent what data it will receive (via placeholders or context), what steps to follow, which tools to consider using, and what information needs to be passed on. The explicit instructions to include original data (e.g., "Crucially: Also include the original name...") are vital for data propagation in chains.
    - `expected_output`: This describes the desired format and content of the task's result. Referencing the Pydantic model (Adhere strictly to the ... Pydantic schema.) helps the LLM generate output that conforms to the required structure.
- Agent Assignment: Each task is assigned to a specific agent (e.g., agent=hr_coordinator).
- Structured Output Enforcement: `output_pydantic=HROutput` (and similar for other tasks) tells Crew AI to use the specified Pydantic model to validate and parse the agent's output for this task. If the agent produces output that doesn't fit the model, Crew AI can try to guide the agent to fix it or raise an error.

In [16]:
# Task 1: HR Processing
# Note: The input dictionary key 'new_hire_details' in crew.kickoff() must match the placeholders used here.
hr_task = Task(
    description=(
        "Process the onboarding for a new hire. Here are their details:\n"
        "- Name: {name}\n"
        "- Email: {email}\n"
        "- Job Title: {job_title}\n"
        "- Start Date: {start_date}\n"
        "- Shipping Address: {shipping_address}\n\n"
        "Your tasks:\n"
        "1. Use the 'create_employee_record_api' tool with the name, email, job title, and start date.\n"
        "2. Extract the new 'employee_id' from the tool's response.\n"
        "3. Use the 'initiate_background_check_api' tool using the extracted employee_id, the original full name, and email.\n"
        "4. Compile the results, including the employee_id and status messages from the tools."
        "5. **Crucially**: Also include the original name, email, job title, start date, and shipping address in your final output, as they are needed by later tasks." # Explicitly ask the agent to pass data forward
    ),
    expected_output=(
        "A structured response containing the 'employee_id', 'hr_record_status', 'background_check_status', "
        "and the original 'original_name', 'original_email', 'original_job_title', 'original_start_date', and 'original_shipping_address'. "
        "Adhere strictly to the HR_Output Pydantic schema."
    ),
    agent=hr_coordinator,
    output_pydantic=HROutput
)

In [17]:
it_task = Task(
    description=(
        "Provision IT resources based on the HR processing results and original hire details provided in the context.\n"
        "You will receive the context containing: 'employee_id', 'original_name', 'original_job_title', and 'original_shipping_address'.\n\n"
        "Your tasks:\n"
        "1. Use the 'provision_account_api' tool with the 'employee_id', 'original_name', and 'original_job_title' from the context.\n"
        "2. Extract the 'username' from the tool response.\n"
        "3. Use the 'assign_hardware_api' tool with the 'employee_id', 'original_job_title', and 'original_shipping_address' from the context.\n"
        "4. Compile the results including account status, username, hardware status, assigned hardware, and shipment tracking ID."
        "5. **Crucially**: Also include the 'employee_id', 'original_name', 'original_job_title', and 'original_start_date' in your final output for the next step." # Pass needed info
    ),
    expected_output=(
        "A structured response containing 'it_account_status', 'username', 'hardware_status', 'assigned_hardware', "
        "'shipment_tracking_id'. It MUST also include 'employee_id', 'original_name', 'original_job_title', and 'original_start_date' from the context/input. "
        "Adhere strictly to the IT_Output Pydantic schema."
    ),
    agent=it_provisioner,
    context=[hr_task], # Depends on the output of the HR task
    output_pydantic=ITOutput
)

In [18]:
# Task 3: Welcome & Training
# Context from it_task (ITOutput) will be available
welcome_task = Task(
    description=(
        "Complete the final onboarding steps using information from the IT provisioning context.\n"
        "You will receive the context containing: 'employee_id', 'username', 'original_name', 'original_job_title', and 'original_start_date'.\n\n"
        "Your tasks:\n"
        "1. Determine a suitable 'first_day_contact'. Use 'Hiring Manager' as the placeholder.\n"
        "2. Use the 'generate_welcome_package' tool with 'employee_id', 'original_name', 'original_job_title', 'original_start_date', the determined 'first_day_contact', and the 'username' from the context.\n"
        "3. Use the 'schedule_training_api' tool with the 'employee_id' and 'original_start_date' from the context.\n"
        "4. Compile the final results including welcome package status/content and training schedule status/details."
        "5. Set the 'final_onboarding_status' field based on whether the welcome package and training scheduling were successful."
    ),
    expected_output=(
        "A structured response containing 'welcome_package_status', 'welcome_email_content' (with subject and body), "
        "'training_schedule_status', 'scheduled_trainings' (list of sessions/datetimes), 'employee_id', 'username', and a 'final_onboarding_status' string. "
        "Adhere strictly to the Welcome_Output Pydantic schema."
    ),
    agent=welcome_coordinator,
    context=[it_task], # Depends on the output of the IT task
    output_pydantic=WelcomeOutput
)

# Crew Definition and Execution

This final section assembles the agents and tasks into a "crew" and then starts the workflow.
- Crew Definition:
    - The Crew object orchestrates the execution of tasks by the agents.
    - `agents=[...]`: A list of all agents participating in this crew.
    - `tasks=[...]`: A list of all tasks to be executed. The order in this list, combined with the context definitions in tasks and the `Process.sequential` setting, determines the execution flow.
    - `process=Process.sequential`: This crucial parameter specifies that the tasks should be executed one after another, in the order they are listed (and as dictated by their context dependencies). The output of one task becomes available to the next.
    - `verbose=True`: Enables verbose output for the crew's operations, showing which task is running, etc. verbose=2 would provide even more detail including the LLM's internal thoughts.
- Input Data:
    - `new_hire`: A Python dictionary containing the initial information about the new employee. The keys in this dictionary (name, email, etc.) match the placeholders used in the description of the first task (`hr_task`).
- Workflow Execution:
    - `onboarding_crew.kickoff(inputs=new_hire)`: This method starts the crew's process.
        - The inputs argument provides the initial data, which is passed to the first task(s) that don't have a context dependency.
        - The crew will then execute `hr_task`, then `it_task` (using output from `hr_task`), and finally `welcome_task` (using output from `it_task`).
    - The `kickoff()` method returns the result of the final task in the sequential process. In this case, it will be the output of `welcome_task`, structured according to `WelcomeOutput` Pydantic model.
- Output Handling:
    - The `final_result` (which is a dictionary representation of the `WelcomeOutput` Pydantic model) is printed.
    - Specific fields from `final_result` are then accessed and printed to demonstrate how to retrieve particular pieces of information from the structured output. Because welcome_task was instructed to include `employee_id` and `username` in its output (and these fields are defined in `WelcomeOutput`), they are available in the `final_result`.

In [19]:
onboarding_crew = Crew(
    agents=[hr_coordinator, it_provisioner, welcome_coordinator],
    tasks=[hr_task, it_task, welcome_task],
    process=Process.sequential, # Defines the sequential execution order
    verbose=True
)

In [22]:
new_hire = {
    "name": "Sodhi Singh",
    "email": "sodhi.singh@example.com",
    "job_title": "Senior Software Engineer",
    "start_date": (datetime.date.today() + datetime.timedelta(days=14)).strftime("%Y-%m-%d"),
    "shipping_address": "123 Main St, Anytown, USA 12345"
}

In [23]:
final_result = onboarding_crew.kickoff(inputs=new_hire)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a4cd663b-51aa-48c0-8b03-4f230a663e48                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 3fa0f3d6-cbbc-4a00-932e-bef83a5aac36
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 3fa0f3d6-cbbc-4a00-932e-bef83a5aac36
       Status: Executing Task...
    └── 🤖 Agent: HR Coordinator
            Status: In Progress

# Agent: HR Coordinator
## Task: Process the onboarding for a new hire. Here are their details:
- Name: Sodhi Singh
- Email: sodhi.singh@example.com
- Job Title: Senior Software Engineer
- Start Date: 2026-01-03
- Shipping Address: 123 Main St, Anytown, USA 12345

Your tasks:
1. Use the 'create_employee_record_api' tool with the name, email, job title, and start date.
2. Extract the new 'employee_id' from the tool's response.
3. Use the 'initiate_background_check_api' tool using the extracted employee_id, the original full name, and email.
4. Compile the results, including the employee_id and status messages from the tools.5. **Crucially**: Also include the original name, email, job title, start date, and shipping address in your final output, as they are needed by later tasks.


🤖 Agent: HR Coordinator
    Status: In Progress

...



# Agent: Welcome and Training Coordinator
## Final Answer: 
{
  "employee_id": "EMP35188",
  "username": "ssingh",
  "welcome_package_status": "Success",
  "welcome_email_content": [{
    "subject": "Welcome to Example Company, Sodhi Singh!",
    "body": "Dear Sodhi Singh,\n\nWe are thrilled to welcome you to Example Company as our new Senior Software Engineer! Your first day is scheduled for 2026-01-03.\n\nPlease plan to arrive by 9:00 AM. Your primary contact for the first day will be Hiring Manager.\n\nYour IT accounts have been set up. Your username is: ssingh. You will receive separate instructions on setting your initial password.\n\nWe have also scheduled some introductory training sessions for you (details below or in a separate calendar invite).\n\nWe look forward to having you on the team!\n\nBest regards,\nExample Company HR"
  }],
  "training_schedule_status": "Success",
  "scheduled_trainings": [
    {
      "session": "HR Policies Onboarding",
      "datetime": "2026-01

🚀 Crew: crew
├── 📋 Task: 3fa0f3d6-cbbc-4a00-932e-bef83a5aac36
│      Assigned to: HR Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: HR Coordinator
│           Status: ✅ Completed
├── 📋 Task: 1198579a-1473-47c1-8fbe-3707b1ad5ec3
│      Assigned to: IT Provisioner
│      Status: ✅ Completed
│   └── 🤖 Agent: IT Provisioner
│           Status: ✅ Completed
└── 📋 Task: ddb819b3-3b6a-4011-89ff-0f12008dfca4
       Status: Executing Task...
    └── 🤖 Agent: Welcome and Training Coordinator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 3fa0f3d6-cbbc-4a00-932e-bef83a5aac36
│      Assigned to: HR Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: HR Coordinator
│           Status: ✅ Completed
├── 📋 Task: 1198579a-1473-47c1-8fbe-3707b1ad5ec3
│      Assigned to: IT Provisioner
│      Status: ✅ Completed
│   └── 🤖 Agent: IT Provisioner
│           Status: ✅ Completed
└── 📋 Task: ddb819b3-3b6a-4011-89ff-0f12008dfca4
       Assigned to: Welcome and Training Coordinator
       Status: ✅ Completed
    └── 🤖 Agent: Welcome and Training Coordinator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ddb819b3-3b6a-4011-89ff-0f12008dfca4                                                                     │
│  Agent: Welcome and Training Coordinator                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a4cd663b-51aa-48c0-8b03-4f230a663e48                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [24]:
print(final_result)

employee_id='EMP35188' username='ssingh' welcome_package_status='Success' welcome_email_content=WelcomeEmailContent(subject='Welcome to Example Company, Sodhi Singh!', body='Dear Sodhi Singh,\n\nWe are thrilled to welcome you to Example Company as our new Senior Software Engineer! Your first day is scheduled for 2026-01-03.\n\nPlease plan to arrive by 9:00 AM. Your primary contact for the first day will be Hiring Manager.\n\nYour IT accounts have been set up. Your username is: ssingh. You will receive separate instructions on setting your initial password.\n\nWe have also scheduled some introductory training sessions for you (details below or in a separate calendar invite).\n\nWe look forward to having you on the team!\n\nBest regards,\nExample Company HR') training_schedule_status='Success' scheduled_trainings=[WelcomeTrainingInfo(session='HR Policies Onboarding', datetime='2026-01-04 10:00 AM'), WelcomeTrainingInfo(session='IT Security Awareness', datetime='2026-01-04 02:00 PM'), Wel

In [25]:
print(f"Final Status: {final_result['final_onboarding_status']}")

Final Status: Onboarding completed successfully.


In [26]:
print(f"Employee ID: {final_result['employee_id']}")
print(f"Username: {final_result['username']}")

Employee ID: EMP35188
Username: ssingh
